In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,asia,9.5506,122.5164,27.40,83,99,8.87,PH,1690323248
1,1,grytviken,-54.2811,-36.5092,-3.48,80,76,4.60,GS,1690323248
2,2,carahue,-38.7000,-73.1667,8.03,94,56,2.03,CL,1690323248
3,3,brisas de zicatela,15.8369,-97.0419,31.29,55,75,3.09,MX,1690323249
4,4,saudarkrokur,65.7461,-19.6394,7.38,90,59,3.39,IS,1690323249


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
cities_map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=1,
    color="City"
)

cities_map

c:\Users\sebat\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

filtered_city_data=city_data_df.loc[city_data_df["Cloudiness"]==0]
filtered_city_data.count()

# Drop any rows with null values
filtered_city_data.dropna(how="any")

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,argo,37.6333,22.7333,30.04,30,0,2.05,GR,1690323249
6,6,afaahiti,-17.7500,-149.2833,27.93,81,0,6.88,PF,1690323249
17,17,kruisfontein,-34.0033,24.7314,10.94,36,0,2.53,ZA,1690323252
18,18,bredasdorp,-34.5322,20.0403,7.35,75,0,1.54,ZA,1690323252
21,21,lata,40.1629,-8.3327,16.71,94,0,1.30,PT,1690323253
...,...,...,...,...,...,...,...,...,...,...
538,538,patacamaya,-17.2333,-67.9167,12.28,13,0,6.69,BO,1690323378
542,542,seydi,39.4816,62.9137,32.53,17,0,4.79,TM,1690323379
557,557,lido di ostia,41.7321,12.2765,26.43,55,0,7.72,IT,1690323382
559,559,pingyin,36.2831,116.4453,24.87,85,0,2.55,CN,1690323382


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
hotel_df=filtered_city_data.copy()
hotel_df=pd.DataFrame(hotel_df.loc[:,["City","Country","Lat","Lng","Humidity",]])
hotel_df["Hotel Name"]=""
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
5,argo,GR,37.6333,22.7333,30,
6,afaahiti,PF,-17.7500,-149.2833,81,
17,kruisfontein,ZA,-34.0033,24.7314,36,
18,bredasdorp,ZA,-34.5322,20.0403,75,
21,lata,PT,40.1629,-8.3327,94,
...,...,...,...,...,...,...
538,patacamaya,BO,-17.2333,-67.9167,13,
542,seydi,TM,39.4816,62.9137,17,
557,lido di ostia,IT,41.7321,12.2765,55,
559,pingyin,CN,36.2831,116.4453,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
argo - nearest hotel: Apollon Hotel
afaahiti - nearest hotel: Omati Lodge
kruisfontein - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
lata - nearest hotel: Residencial Botânico
tazacorte - nearest hotel: App Leyma
hamilton - nearest hotel: North Vista Manor
kodiak - nearest hotel: Shelikof Lodge
kolonia - nearest hotel: Wasserturm Hotel Cologne
kill devil hills - nearest hotel: Mariner Days Inn & Suites
sho'rchi - nearest hotel: Идеал СС
talnakh - nearest hotel: Талнах
port elizabeth - nearest hotel: Waterford Hotel
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
mentor-on-the-lake - nearest hotel: Best Western
demopolis - nearest hotel: No hotel found
nanao - nearest hotel: ホテル・アルファ－ワン能登和倉
port lincoln - nearest hotel: Boston Hotel
emerald - nearest hotel: Emerald Central Hotel
arzgir - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
tir pul - nearest hotel: No hotel found
maceio - nearest hotel: Maceió Hos

,City,Country,Lat,Lng,Humidity,Hotel Name
5,argo,GR,37.6333,22.7333,30,Apollon Hotel
6,afaahiti,PF,-17.7500,-149.2833,81,Omati Lodge
17,kruisfontein,ZA,-34.0033,24.7314,36,No hotel found
18,bredasdorp,ZA,-34.5322,20.0403,75,Victoria Hotel
21,lata,PT,40.1629,-8.3327,94,Residencial Botânico
...,...,...,...,...,...,...
538,patacamaya,BO,-17.2333,-67.9167,13,Hostal Viña del Mar
542,seydi,TM,39.4816,62.9137,17,No hotel found
557,lido di ostia,IT,41.7321,12.2765,55,Airport Palace
559,pingyin,CN,36.2831,116.4453,85,No hotel found


In [7]:
# Configure the map plot
hotel_map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["Hotel Name","Country"]
)

#Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)